# 首先計算除了neg_word以外的情緒得分

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
#read sentiment_pred.csv
sentiment_pred = pd.read_csv('sentiment_pred.csv', encoding = 'ISO-8859-1')

In [ ]:
#rename
df = sentiment_pred.rename(columns = {'Unnamed: 0' : 'relationword'})

In [ ]:
#get required columns
df_word = df.iloc[:, 0]
df_sentiment = df.iloc[:, -1]
#rename
df_sentiment.columns = ['sentiment']
#concat
df = pd.concat([df_word, df_sentiment], axis = 1)

In [ ]:
#讀入 step2 做出來的 relation_word.csv
relation_word = pd.read_csv('relation_word.csv', encoding='ISO-8859-1')

In [ ]:
#五大特質個別的 target_word list
food_word_list = ['lunch','delicious','menu','fresh','breakfast','drink','drinks','tasty','dinner','selection','meal','taste','flavor','dish','food']
atmosphere_word_list = ['atmosphere','happy','fun','friendly']
service_word_list = ['order','ordered','staff','service','wait','server','servered','quality']
price_word_list = ['price','prices']
place_word_list = ['place','places','location','area']

In [ ]:
#create a new list
targetword2 = []
#如果relation_word中 targetword欄位裡的 word，出現在五大特質 target_word list中，將其加入至 targetword2中
targetword2 = []
for word in relation_word['targetword']:
    if word in food_word_list:
        targetword2.append('food')
    elif word in atmosphere_word_list:
        targetword2.append('atmosphere')
    elif word in service_word_list:
        targetword2.append('service')
    elif word in price_word_list:
        targetword2.append('price')
    elif word in place_word_list:
        targetword2.append('place')
    
#len(targetword2)

In [ ]:
#將 targetword2轉成 dataframe後，再將其新增成 relation_word的新欄位
relation_word['targetword'] = pd.DataFrame(targetword2, columns = ['targetword'])

In [ ]:
#將 relationword 中的 business_id和 targetword欄位合併再一起，並且新增成 relation_word的新欄位
relation_word['id_targetword'] = relation_word['business_id'] + '*' + relation_word['targetword']

In [ ]:
#get required columns
relation_word2 = pd.concat([relation_word.iloc[:, -1], relation_word.iloc[:, -2]], axis = 1)

In [ ]:
#outer join 
merge_df = pd.merge(relation_word2, df, how = 'outer')
#fillna with 0
merge_df = merge_df.fillna(0)
merge_df = merge_df.iloc[0 : 193677, :]
#merge_df

In [ ]:
#group by with column id_targetword
re = merge_df.groupby(['id_targetword'])
#test
re.get_group('EJUaeaUcedPJd769_nG1pA*food')['sentiment'].sum()

In [ ]:
#create two new lists
business_id = []
final_output = []

In [ ]:
#把 business_id 和計算出的各家店在五大特質的情緒得分分別 append進去list當中
for business in merge_df['id_targetword']:
    business_id.append(business)
    score = re.get_group(business)['sentiment'].sum()
    final_output.append(score)

In [ ]:
#list to dataframe
df_business_id = pd.DataFrame(business_id, columns = ['id_targetword'])
df_final_output = pd.DataFrame(final_output, columns = ['score'])

In [ ]:
#concat
df_output = pd.concat([df_business_id, df_final_output], axis = 1)
#drop duplicates
df_output = df_output.drop_duplicates()

* Series.str.split(self, pat=None, n=-1, expand=False
    - Split strings around given separator/delimiter.
    - Splits the string in the Series/Index from the beginning, at the specified delimiter string. Equivalent to str.split().

#### parameters
* pat : str, optional
    - String or regular expression to split on. If not specified, split on whitespace.

* n : int, default -1 (all)
    - Limit number of splits in output. None, 0 and -1 will be interpreted as return all splits.

* expand : bool, default False
    - Expand the splitted strings into separate columns.

In [ ]:
#把 id_targetword中的 business_id 和 targetword分開
df_1 = df_output['id_targetword'].str.split('*', expand = True)
#rename
df_1.columns = ['business_id', 'targetword']

In [ ]:
#合併 df_1 以及情緒得分欄位
df_output2 = pd.concat([df_1, df_output['score']], axis = 1)
#df_output2.head(5)

In [ ]:
#series to list
list_output_id = df_output2['business_id'].to_list()
list_output_target = df_output2['targetword'].to_list()
list_output_score = df_output2['score'].to_list()

In [ ]:
#zip
list_word_score = zip(list_output_target, list_output_score)
df_output3 = list(zip(list_output_id, list_word_score))

In [ ]:
#取出不重複的 business_id
businessid_df = df_output2['business_id']
noduplicate = businessid_df.drop_duplicates()

## 分別把各家餐廳在五大特質的情緒得分做成dataframe

In [ ]:
atmosphere = []
atmosphere_word_list = ['atmosphere','happy','fun','friendly']
for i in range(len(df_output3)):
    if df_output3[i][1][0] in atmosphere_word_list:
        atmosphere.append(int(df_output3[i][1][1]))
    else:
        atmosphere.append(0)

df_atmosphere = pd.DataFrame(atmosphere)
df_atmosphere.columns=['atmosphere']


idx = businessid_df.index
df_atmosphere.index = idx

atmosphere_df = pd.concat([businessid_df, df_atmosphere], axis=1)

In [ ]:
food = []
food_word_list = ['lunch','delicious','menu','fresh','breakfast','drink','drinks','tasty','dinner','selection','meal','taste','flavor','dish','food']
for i in range(len(df_output3)):
    if df_output3[i][1][0] in food_word_list:
        food.append(int(df_output3[i][1][1]))
    else:
        food.append(0)

df_food = pd.DataFrame(food)
df_food.columns=['food']

df_food.index = idx

food_df = pd.concat([businessid_df, df_food], axis=1)

In [ ]:
service = []

service_word_list = ['order','ordered','staff','service','wait','server','servered','quality']
for i in range(len(df_output3)):
    if df_output3[i][1][0] in service_word_list:
        service.append(int(df_output3[i][1][1]))
    else:
        service.append(0)

df_service = pd.DataFrame(service)
df_service.columns=['service']
df_service.index = idx

service_df = pd.concat([businessid_df, df_service], axis=1)

In [ ]:
price = []

price_word_list = ['price','prices']
for i in range(len(df_output3)):
    if df_output3[i][1][0] in price_word_list:
        price.append(int(df_output3[i][1][1]))
    else:
        price.append(0)
        
df_price = pd.DataFrame(price)
df_price.columns=['price']

df_price.index = idx

price_df = pd.concat([businessid_df, df_price], axis=1)

In [ ]:
place = []

place_word_list = ['place','places','location','area']
for i in range(len(df_output3)):
    if df_output3[i][1][0] in place_word_list:
        place.append(int(df_output3[i][1][1]))
    else:
        place.append(0)

df_place = pd.DataFrame(place)
df_place.columns=['place']

df_place.index = idx

place_df = pd.concat([businessid_df, df_place], axis=1)

In [ ]:
#把這五大特質合併在一起
final = pd.concat([atmosphere_df, food_df['food'], service_df['service'], price_df['price'], place_df['place']], axis = 1)

In [ ]:
#group by with business_id
fi = final.groupby('business_id')
#create a new dict
final_output = {}
for i in final['business_id']:
    a_score = list(fi.get_group(i).sum())[1:]
    final_output.update({i:a_score})

#test    
#fi.get_group('9yKzy9PApeiPPOUJEtnvkg').sum()    

In [ ]:
#dict to dataframe
final_T = pd.DataFrame.from_dict(final_output, orient='index', columns=['atmosphere','food','service','price','place'])

In [ ]:
#series to dataframe
noduplicate = pd.DataFrame(noduplicate, columns = ['business_id'])

In [ ]:
#concat
id2x = noduplicate.index
final_T.index = id2x
final_score = pd.concat([noduplicate, final_T], axis = 1)

In [ ]:
#read total_id.csv
total_id = pd.read_csv('total_id.csv',encoding='ISO-8859-1')
total_business_id = total_id['business_id']

In [ ]:
#merge and fillna with 0
final_all = pd.merge(total_business_id, final_score, how='outer')
final_all = final_all.fillna(0)
#final_all

# neg_word

In [ ]:
#read sentiment_neg_pred.csv
sentiment_pred = pd.read_csv('sentiment_neg_pred.csv', encoding='ISO-8859-1') 

In [ ]:
#rename
df = sentiment_pred.rename(columns={'Unnamed: 0': 'relationword'})

In [ ]:
#get required columns
df_word = df.iloc[:, 0]
df_sentiment = df.iloc[:, -1]
#rename
df_sentiment.columns = ['sentiment']
#concat
df = pd.concat([df_word, df_sentiment], axis = 1)

In [ ]:
#讀入 step2 做出來的 relation_neg_word.csv
relation_word = pd.read_csv('relation_neg_word.csv', encoding='ISO-8859-1')

In [ ]:
#五大特質個別的 target_word list
food_word_list = ['lunch','delicious','menu','fresh','breakfast','drink','drinks','tasty','dinner','selection','meal','taste','flavor','dish','food']
atmosphere_word_list = ['atmosphere','happy','fun','friendly']
service_word_list = ['order','ordered','staff','service','wait','server','servered','quality']
price_word_list = ['price','prices']
place_word_list = ['place','places','location','area']

In [ ]:
#create a new list
targetword2 = []
#如果relation_word中 targetword欄位裡的 word，出現在五大特質 target_word list中，將其加入至 targetword2中
targetword2 = []
for word in relation_word['targetword']:
    if word in food_word_list:
        targetword2.append('food')
    elif word in atmosphere_word_list:
        targetword2.append('atmosphere')
    elif word in service_word_list:
        targetword2.append('service')
    elif word in price_word_list:
        targetword2.append('price')
    elif word in place_word_list:
        targetword2.append('place')
    
#len(targetword2)

In [ ]:
#將 relation_word 中的 targetword欄位轉成 dataframe後取出
targetwordnew = pd.DataFrame(list(relation_word['targetword']), columns=['relationword'])
#rename
relation_word = relation_word.rename(columns={'relationword': 'neg_word'})

In [ ]:
#將 targetword2轉成 dataframe後，再將其新增成 relation_word的新欄位
relation_word['targetword'] = pd.DataFrame(targetword2, columns=['targetword'])
#去除 relation_word中的 NA值
relation_word = relation_word.dropna()

In [ ]:
#合併 relation_word和 targetwordnew
relation_word2 = pd.concat([relation_word, targetwordnew],axis=1)
#去除 relation_word2中的 NA值
relation_word2 = relation_word2.dropna()

In [ ]:
#將 relation_word2 中的 business_id和 targetword欄位合併再一起，並且新增成 relation_word2的新欄位
relation_word2['id_targetword'] = relation_word2['business_id'] + '*' + relation_word2['targetword']
#relation_word2

In [ ]:
#get required columns and dropna
relation_word3 = pd.concat([relation_word2.iloc[:,-1:], relation_word2.iloc[:,-3:-2], relation_word2.iloc[:,-2:-1]],axis=1)
relation_word3 = relation_word3.dropna()
#relation_word3

In [ ]:
# outer join 
merge_df = pd.merge(relation_word3, df, how='outer')
# dropna
merge_df = merge_df.dropna()
merge_df

In [ ]:
#group by with id_targetword
re = merge_df.groupby(['id_targetword'])

In [ ]:
#create two new lists
business_id = []
final_output = []

In [ ]:
#把 business_id 和計算出的各家店在五大特質的情緒得分分別 append進去list當中，並且乘上 -1做校正
for business in merge_df['id_targetword']:
    business_id.append(business)
    score = (re.get_group(business)['sentiment']*-1).sum()
    final_output.append(score)

In [ ]:
#list to dataframe
df_business_id = pd.DataFrame(business_id, columns=['id_targetword'])
df_final_output = pd.DataFrame(final_output, columns=['score'])

In [ ]:
#concat
df_output = pd.concat([df_business_id, df_final_output], axis = 1)
#drop duplicates
df_output = df_output.drop_duplicates()

In [ ]:
#把 id_targetword中的 business_id 和 targetword分開
df_1 = df_output['id_targetword'].str.split('*', expand = True)
#rename
df_1.columns = ['business_id', 'targetword']

In [ ]:
#合併 df_1 以及情緒得分欄位
df_output2 = pd.concat([df_1, df_output['score']],axis=1)

In [ ]:
#series to list
list_output_id = df_output2['business_id'].to_list()
list_output_target = df_output2['targetword'].to_list()
list_output_score = df_output2['score'].to_list()

In [ ]:
#zip
list_word_score = zip(list_output_target, list_output_score)
df_output3 = list(zip(list_output_id, list_word_score))

In [ ]:
#取出不重複的 business_id
businessid_df = df_output2['business_id']
noduplicate = businessid_df.drop_duplicates()

## 分別把各家餐廳在五大特質的情緒得分做成dataframe

In [ ]:
atmosphere = []
atmosphere_word_list = ['atmosphere','happy','fun','friendly']
for i in range(len(df_output3)):
    if df_output3[i][1][0] in atmosphere_word_list:
        atmosphere.append(int(df_output3[i][1][1]))
    else:
        atmosphere.append(0)

df_atmosphere = pd.DataFrame(atmosphere)
df_atmosphere.columns=['atmosphere']

idx = businessid_df.index
df_atmosphere.index = idx

atmosphere_df = pd.concat([businessid_df, df_atmosphere], axis=1)

In [ ]:
food = []
food_word_list = ['lunch','delicious','menu','fresh','breakfast','drink','drinks','tasty','dinner','selection','meal','taste','flavor','dish','food']
for i in range(len(df_output3)):
    if df_output3[i][1][0] in food_word_list:
        food.append(int(df_output3[i][1][1]))
    else:
        food.append(0)

df_food = pd.DataFrame(food)
df_food.columns=['food']

df_food.index = idx

food_df = pd.concat([businessid_df, df_food], axis=1)

In [ ]:
service = []

service_word_list = ['order','ordered','staff','service','wait','server','servered','quality']
for i in range(len(df_output3)):
    if df_output3[i][1][0] in service_word_list:
        service.append(int(df_output3[i][1][1]))
    else:
        service.append(0)

df_service = pd.DataFrame(service)
df_service.columns=['service']

df_service.index = idx

service_df = pd.concat([businessid_df, df_service], axis=1)

In [ ]:
price = []

price_word_list = ['price','prices']
for i in range(len(df_output3)):
    if df_output3[i][1][0] in price_word_list:
        price.append(int(df_output3[i][1][1]))
    else:
        price.append(0)

df_price = pd.DataFrame(price)
df_price.columns=['price']

df_price.index = idx

price_df = pd.concat([businessid_df, df_price], axis=1)

In [ ]:
place = []

place_word_list = ['place','places','location','area']
for i in range(len(df_output3)):
    if df_output3[i][1][0] in place_word_list:
        place.append(int(df_output3[i][1][1]))
    else:
        place.append(0)

df_place = pd.DataFrame(place)
df_place.columns=['place']

df_place.index = idx

place_df = pd.concat([businessid_df, df_place], axis=1)

In [ ]:
#把這五大特質合併在一起
final = pd.concat([atmosphere_df, food_df['food'], service_df['service'], price_df['price'], place_df['place']], axis=1)

In [ ]:
#group by with business_id
fi = final.groupby('business_id')
#create a new dict
final_output = {}
for i in final['business_id']:
    a_score = list(fi.get_group(i).sum())[1:]
    final_output.update({i:a_score})

#test
#fi.get_group('9yKzy9PApeiPPOUJEtnvkg').sum() 

In [ ]:
#dict to dataframe
final_T = pd.DataFrame.from_dict(final_output, orient='index', columns=['atmosphere','food','service','price','place'])

In [ ]:
#series to dataframe
noduplicate = pd.DataFrame(noduplicate, columns=['business_id']) 

In [ ]:
#concat
id2x = noduplicate.index
final_T.index = id2x
final_score = pd.concat([noduplicate,final_T],axis = 1)

In [ ]:
#read total_id.csv
total_id = pd.read_csv('total_id.csv',encoding='ISO-8859-1')
total_business_id = total_id['business_id']

In [ ]:
#merge and fillna with 0
final_neg = pd.merge(total_business_id, final_score, how='outer')
final_neg = final_neg.fillna(0)
#final_neg

In [ ]:
#合併 final_all跟 final_neg
final_all_score = final_all.iloc[:,0:] + final_neg.iloc[:,0:]
#drop_duplicates and dropna
final_all_score = final_all_score.drop_duplicates().dropna()

In [ ]:
#算出五大特質的平均分數
final_all_score['overall'] = (final_all_score['atmosphere'] + final_all_score['food'] + 
                              final_all_score['service'] + final_all_score['price'] + final_all_score['place'])/5
final_all_score.dropna()

In [ ]:
#export
final_all_score.to_csv('final.csv', encoding='utf-8')